# Multilayer Perceptron (MLP) for Battery Management System (BMS) SOH Estimation

<img src="../../doc/img/MachineLearningNetwork.png" height="1080" width="1920"
     alt="Machine Learning Network"
     style="fit: left; margin-right: 10px;"  />

# Importing Libraries

In [28]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Activation 
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (C:\Users\LucasTroy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend\__init__.py)

# Import Data

In [29]:
# Import Data
df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
df.describe()

,V,I,SOC,T_surf,SOH
count,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06
mean,3.471013e+00,3.610140e+00,4.188879e+01,5.180549e+01,8.566437e+01
std,8.445745e-01,2.830966e+01,4.022232e+01,2.451698e+01,9.026790e+00
min,-9.394656e-01,-5.500295e+01,-5.753759e+00,2.499965e+01,7.000000e+01
25%,3.144859e+00,-1.900000e+01,1.230206e+00,2.583835e+01,7.800000e+01
50%,3.693897e+00,5.000000e+00,3.474387e+01,4.826449e+01,8.600000e+01
75%,4.064538e+00,2.600000e+01,8.456010e+01,7.304868e+01,9.400000e+01
max,4.723157e+00,5.500289e+01,1.070423e+02,1.015903e+02,1.000000e+02


# Data Preprocessing

We have to add new columns to the dataset that are the previous values of V, I and T. This will allow our model to detect the trend/gradient of the data.

In [27]:
# Data Preprocessing
# Apply the shift to the entire dataframe, but only on each unique SOH, I Pair
# Define the columns to shift
cols_to_shift = ['V', 'I', 'T_surf']

# Apply the shift operation to each group
for col in cols_to_shift:
    df[col + '-1'] = df.groupby(['SOH', 'I'])[col].shift(1)

# Drop rows with NaN values (optional)
# df = df.dropna()

df

KeyboardInterrupt: 

# Model Training

In [2]:
# Model Training

model = Sequential([ 
	
	# Flatten(input_shape=(df.shape)), 
	
	# dense layer 1 
	Dense(256, activation='relu'), 
	
	# dense layer 2 
	Dense(128, activation='relu'), 
	
	# output layer 
	Dense(10, activation='relu'), 
]) 


# Model Testing

In [3]:
# Model Testing
model.compile(optimizer='adam', 
			loss='sparse_categorical_crossentropy', 
			metrics=['accuracy']) 

# define training variables V, I, and T here: 
X = df['V', 'I', 'T_surf', 'V-1', 'I-1', 'T_surf-1']
Y = df['SOH']

# split up the data into training and testing 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model.fit(x_train, y_train, epochs=10, 
		batch_size=2000, 
		validation_split=0.2) 

results = model.evaluate(x_test, y_test, verbose = 0) 
print('test loss, test acc:', results)


# Model Export

In [4]:
# Model Export
# save Keras model
model.save("model_file_name" +'.h5')
# convert Keras model to a tflite model 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open("model_file_name" + '.tflite', 'wb') as f:
    f.write(tflite_model)
	